# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,12.66,68,52,7.82,SH,1728497479
1,1,bethel,41.3712,-73.4140,18.52,46,20,5.14,US,1728497241
2,2,kruisfontein,-34.0033,24.7314,17.29,82,3,1.33,ZA,1728497481
3,3,whitehorse,60.7161,-135.0538,2.12,91,100,4.02,CA,1728497281
4,4,san luis de la loma,17.2706,-100.8939,28.32,76,70,2.19,MX,1728497484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='City',
    alpha=0.6,
    frame_width=700,
    frame_height=500,          
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 18) & 
    (city_data_df['Max Temp'] < 25) & 
    (city_data_df['Wind Speed'] < 4) & 
    (city_data_df['Cloudiness'] < 20) &
    (city_data_df['Humidity'] < 60)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(how='any')

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,wildwood,38.5828,-90.6629,24.67,37,0,2.68,US,1728497516
143,143,ataq,14.5377,46.8319,24.98,44,0,3.45,YE,1728497662
157,157,kalengwa,-13.4659,25.0027,21.87,22,3,2.55,ZM,1728497678
180,180,peru,40.7537,-86.0689,22.34,37,0,1.54,US,1728497528
195,195,gumdag,39.2061,54.5906,21.17,48,0,2.17,TM,1728497519
283,283,mount isa,-20.7333,139.5000,24.87,20,0,3.09,AU,1728497839
321,321,hamilton,39.1834,-84.5333,23.47,40,0,2.57,US,1728497298
339,339,al qaryatayn,34.2296,37.2407,23.32,29,0,1.21,SY,1728497834
356,356,oltinko'l,40.8011,72.1634,21.27,35,0,1.62,UZ,1728497929
363,363,chulakkurgan,43.7650,69.1814,18.54,23,0,3.98,KZ,1728497939


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,wildwood,US,38.5828,-90.6629,37,
143,ataq,YE,14.5377,46.8319,44,
157,kalengwa,ZM,-13.4659,25.0027,22,
180,peru,US,40.7537,-86.0689,37,
195,gumdag,TM,39.2061,54.5906,48,
283,mount isa,AU,-20.7333,139.5000,20,
321,hamilton,US,39.1834,-84.5333,40,
339,al qaryatayn,SY,34.2296,37.2407,29,
356,oltinko'l,UZ,40.8011,72.1634,35,
363,chulakkurgan,KZ,43.7650,69.1814,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wildwood - nearest hotel: Wildwood Hotel
ataq - nearest hotel: الشارقة بلازا
kalengwa - nearest hotel: No hotel found
peru - nearest hotel: No hotel found
gumdag - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
hamilton - nearest hotel: North Vista Manor
al qaryatayn - nearest hotel: No hotel found
oltinko'l - nearest hotel: No hotel found
chulakkurgan - nearest hotel: No hotel found
dalbandin - nearest hotel: SF Al-dawood Restaurant
fort morgan - nearest hotel: Hampton Inn Fort Morgan
cortez - nearest hotel: BEST WESTERN Turquoise Inn & Suites
ceerigaabo - nearest hotel: Sanaag Hotel
farah - nearest hotel: No hotel found
az zubayr - nearest hotel: فندق برج بابل
balkanabat - nearest hotel: Nebitçi
bhimbar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,wildwood,US,38.5828,-90.6629,37,Wildwood Hotel
143,ataq,YE,14.5377,46.8319,44,الشارقة بلازا
157,kalengwa,ZM,-13.4659,25.0027,22,No hotel found
180,peru,US,40.7537,-86.0689,37,No hotel found
195,gumdag,TM,39.2061,54.5906,48,No hotel found
283,mount isa,AU,-20.7333,139.5000,20,Ibis Styles
321,hamilton,US,39.1834,-84.5333,40,North Vista Manor
339,al qaryatayn,SY,34.2296,37.2407,29,No hotel found
356,oltinko'l,UZ,40.8011,72.1634,35,No hotel found
363,chulakkurgan,KZ,43.7650,69.1814,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='City',
    alpha=0.6,
    frame_width=700,
    frame_height=500,
    hover_cols=['Humidity', 'Hotel Name', 'Country']  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)